In [8]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np
from tempfile import TemporaryFile
import os
import pickle
import random 
import operator
import math
import numpy as np
import librosa
import pandas as pd

In [9]:
directory = "D:\Machine Learning\Audio Classifier\\genres\\"
dataset = []
i=0
for folder in os.listdir(directory):
    i+=1
    if i==11 :
        break   
    for file in os.listdir(directory+folder):  
        (audio,rate) = librosa.load(directory+folder+"/"+file)
        mfcc_feat = librosa.feature.mfcc(y=audio,sr= rate, n_mfcc=40)
        mean_matrix = np.mean(mfcc_feat.T, axis=0)
        dataset.append([mean_matrix,i])
        

In [11]:
features_df= pd.DataFrame(dataset, columns=['features', 'class'])
features_df

,features,class
0,"[-113.57065, 121.57179, -19.168142, 42.36642, ...",1
1,"[-207.5017, 123.991264, 8.955127, 35.877647, 2...",1
2,"[-90.722595, 140.4463, -29.09389, 31.684334, -...",1
3,"[-199.5442, 150.0909, 5.6626782, 26.85908, 1.7...",1
4,"[-160.3377, 126.219635, -35.58781, 22.148071, ...",1
...,...,...
995,"[-153.64096, 109.86465, -23.065697, 59.050125,...",10
996,"[-142.39203, 116.18963, -32.147167, 49.11784, ...",10
997,"[-124.95227, 115.177155, -47.97515, 52.814674,...",10
998,"[-225.00775, 123.64677, -9.7241745, 56.605164,...",10


In [22]:
df = features_df.sample(frac=1).reset_index(drop=True)
df.head()

,features,class
0,"[-76.7762, 98.37953, -5.0335135, 49.34434, -10...",5
1,"[-182.20235, 83.476616, 4.8968363, 12.707844, ...",9
2,"[-169.1751, 110.45037, -8.545039, 43.8886, 0.3...",9
3,"[-248.96225, 139.03177, -16.240025, 57.11619, ...",1
4,"[-222.9824, 102.862144, -1.6518362, 45.21669, ...",6


In [17]:
X= np.array(df['features'].tolist())
y= np.array(df['class'].tolist())

In [55]:
y= np.array(pd.get_dummies(y))
y.shape

(1000, 10)

In [56]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2 , random_state=0)


In [57]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [59]:
num_label=y.shape[1]
num_label

10

In [60]:
model= Sequential()

model.add(Dense(100, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(10))
model.add(Activation('softmax'))


In [61]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 100)               4100      
_________________________________________________________________
activation_11 (Activation)   (None, 100)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 200)               20200     
_________________________________________________________________
activation_12 (Activation)   (None, 200)               0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 100)              

In [62]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')

In [63]:
X_train.shape

(800, 40)

In [68]:

from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime


num_epochs = 100
num_batch_size = 32
#checkpointer = ModelCheckpoint(filepath='save_models\audio_classification.hdf5', verbose=1 , save_best_only=True)
start= datetime.now()
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test) )

duration= datetime.now()-start
print("Training completed in time: ", duration)


Epoch 1/100
25/25 [==============================] - 0s 6ms/step - loss: 0.9752 - accuracy: 0.6488 - val_loss: 1.4485 - val_accuracy: 0.5950
Epoch 2/100
25/25 [==============================] - 0s 4ms/step - loss: 0.9621 - accuracy: 0.6650 - val_loss: 1.4568 - val_accuracy: 0.6050
Epoch 3/100
25/25 [==============================] - 0s 4ms/step - loss: 0.9649 - accuracy: 0.6725 - val_loss: 1.4975 - val_accuracy: 0.5950
Epoch 4/100
25/25 [==============================] - 0s 5ms/step - loss: 0.9309 - accuracy: 0.6750 - val_loss: 1.5432 - val_accuracy: 0.6000
Epoch 5/100
25/25 [==============================] - 0s 5ms/step - loss: 0.9155 - accuracy: 0.6662 - val_loss: 1.4857 - val_accuracy: 0.5750
Epoch 6/100
25/25 [==============================] - 0s 5ms/step - loss: 0.8965 - accuracy: 0.6637 - val_loss: 1.5527 - val_accuracy: 0.5850
Epoch 7/100
25/25 [==============================] - 0s 5ms/step - loss: 0.9038 - accuracy: 0.6900 - val_loss: 1.5672 - val_accuracy: 0.5800
Epoch 8/100
2

Epoch 59/100
25/25 [==============================] - 0s 4ms/step - loss: 0.8395 - accuracy: 0.6850 - val_loss: 1.7574 - val_accuracy: 0.5900
Epoch 60/100
25/25 [==============================] - 0s 4ms/step - loss: 0.8192 - accuracy: 0.6913 - val_loss: 1.7954 - val_accuracy: 0.5300
Epoch 61/100
25/25 [==============================] - 0s 4ms/step - loss: 0.8140 - accuracy: 0.7063 - val_loss: 1.7185 - val_accuracy: 0.6000
Epoch 62/100
25/25 [==============================] - 0s 4ms/step - loss: 0.7575 - accuracy: 0.7312 - val_loss: 1.8202 - val_accuracy: 0.5850
Epoch 63/100
25/25 [==============================] - 0s 4ms/step - loss: 0.7966 - accuracy: 0.7138 - val_loss: 1.7128 - val_accuracy: 0.6100
Epoch 64/100
25/25 [==============================] - 0s 4ms/step - loss: 0.8003 - accuracy: 0.7050 - val_loss: 1.8078 - val_accuracy: 0.5900
Epoch 65/100
25/25 [==============================] - 0s 4ms/step - loss: 0.7858 - accuracy: 0.7462 - val_loss: 1.7443 - val_accuracy: 0.5900
Epoch 

In [69]:
test_accuracy=  model.evaluate(X_test, y_test, verbose=0)
print(test_accuracy[1])

0.6100000143051147
